# Prepare data

Importing COVID-19 Data

In [1]:
import json

with open('Resources/dpc-covid19-ita-province.json') as f:
    d = json.load(f)

Importing NetworkX and Pandas

In [52]:
import networkx as nx
import pandas as pd
import numpy as np
import math as mt

Cleaning data

In [3]:
# Create a DataFrame with COVID data, we need just some columns
city_dataframe = pd.DataFrame(d)[['denominazione_provincia', 'lat', 'long']].drop_duplicates()

print("Dataframe contains " + str(city_dataframe.count()[0]) + " rows")

# Remove data having latitude = 0 or longitude = 0 or provincia = "In fase di definizione/aggiornamento"
city_dataframe.drop(city_dataframe[(city_dataframe['lat'] == 0) | \
                                   (city_dataframe['long'] == 0) | \
                                   (city_dataframe['denominazione_provincia'] == 'In fase di definizione/aggiornamento') \
                                  ].index, inplace = True)

city_dataframe.reset_index(drop = True, inplace = True)
                        
print("After removing unusable data, Dataframe contains " + str(city_dataframe.count()[0]) + " rows")

Dataframe contains 108 rows
After removing unusable data, Dataframe contains 107 rows


In [4]:
city_dataframe

,denominazione_provincia,lat,long
0,Chieti,42.351032,14.167546
1,L'Aquila,42.351222,13.398438
2,Pescara,42.464584,14.213648
3,Teramo,42.658918,13.704400
4,Matera,40.667512,16.597924
...,...,...,...
102,Rovigo,45.071073,11.790070
103,Treviso,45.667546,12.245074
104,Venezia,45.434905,12.338452
105,Verona,45.438390,10.993527


# Algorithms
We are not explicitly adding nodes to graph $\Rightarrow$ nodes without any edge will not be put in the graph
## 1. Iteration over all couples $\rightarrow$ Cost: $\mathcal{\Theta}\ (\ n^2\ )$

In [5]:
def allCoupleEdges(graph, dataframe, radius):
    # O (n)
    for i in dataframe.index:

        # O (n)
        for j in dataframe.index:
            if i != j and \
               dataframe.iloc[i, 1] - radius <= dataframe.iloc[j, 1] and \
               dataframe.iloc[i, 1] + radius >= dataframe.iloc[j, 1] and \
               dataframe.iloc[i, 2] - radius <= dataframe.iloc[j, 2] and \
               dataframe.iloc[i, 2] + radius >= dataframe.iloc[j, 2]:
                graph.add_edge(dataframe.iloc[i, 0], dataframe.iloc[j, 0])                        

## 2. Binary search on ordered dataframe $\rightarrow$ Cost: $\mathcal{\Theta}\ (\ n \cdot \log{} n\ )$

### Utility function
Given a dataframe with:
 - Column 0 $\rightarrow$ ID
 - Column 1 $\rightarrow$ Position  
 
Returns a set of all ID couples within *radius* distance

In [6]:
def binarySearchSingle(dataframe, radius):
    # Edges between near cities basing on x position
    # Use of dictionary, in this way search of an element costs O(1)
    edges = {}
    
    # Sort dataframe basing on position O (n log n) using quicksort
    # Use of tmpDataframe to leave dataframe as received
    tmpDataframe = dataframe.sort_values(by = dataframe.columns[1])
    tmpDataframe.reset_index(drop = True, inplace = True)
    
    # O(n)
    for i in tmpDataframe.index:        
        # Set pointers to be used in iterative binary search
        first = 0
        # We just check the left half because we do not need double couples (a, b) and (b, a).
        last = i - 1
        found = False

        # O (log n)
        while first <= last and not found:
            midpoint = (first + last) // 2
        
            # Check if element at midpoint position is near enough            
            if tmpDataframe.iloc[i, 1] - radius <= tmpDataframe.iloc[midpoint, 1]:
                
                # If element at midpoint position is the leftmost element within radius distance
                # i.e. element at (midpoint - 1) position is too far
                if midpoint == 0 or tmpDataframe.iloc[i, 1] - radius > tmpDataframe.iloc[midpoint - 1, 1]:
                
                    # We add to edges all couples composed by (element at i position, element at j position)
                    # for all j from midpoint to i (excluded)
                    edges.update([((tmpDataframe.iloc[i, 0], tmpDataframe.iloc[j, 0]), None) 
                                  for j in range(midpoint, i)])
                    found = True
                
                # Otherwise (element at (midpoint - 1) position is near enough)
                # We search in left half
                else:
                    last = midpoint - 1             
                    
            # Otherwise we must search in right half
            else:
                first = midpoint + 1
                
    return edges

In [7]:
def binarySearchEdges(graph, dataframe, radius):
    xEdges = binarySearchSingle(dataframe, radius)
    yEdges = binarySearchSingle(dataframe.iloc[:, 0::2],  radius)
            
    # O(n)
    for k in xEdges.keys():
        # Searching both for (a,b) and (b,a)
        # O(1)
        if k in yEdges or k[::-1] in yEdges:
            graph.add_edge(*k)

# 1. Create Graph of cities
Build the graph of provinces P using NetworkX. Each node corresponds to a city and two cities a and b are connected by an edge if the following holds: if x,y is the position of a, then b is in position z,w with z in [x-d,x+d] and w in [y-d, y+d], with d=0.8. The graph is symmetric. Use the latitude and longitude information available in the files to get the position of the cities. This task can be done in several ways. Use the one you think is more efficient.

## Set up variables

In [8]:
P_all_couples = nx.Graph()
P = nx.Graph()
radius = 0.8

## Create Graph using algorithm # 1

In [9]:
%%timeit
P_all_couples.clear()
allCoupleEdges(P_all_couples, city_dataframe, radius)

323 ms ± 6.56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Create Graph using algorithm # 2

In [10]:
%%timeit
P.clear()
binarySearchEdges(P, city_dataframe, radius)

49.5 ms ± 646 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Checking results of algorithms

In [11]:
if P.nodes != P_all_couples.nodes:
    raise Exception("P.nodes != P_all_couples.nodes")
    
if P.edges != P_all_couples.edges:
    raise Exception("P.edges != P_all_couples.edges")

# 2. Create graph of point
Generate 2000 pairs of double (x,y) with x in [30,50) and y in [10,20). Repeat the algorithm at step 1, building a graph R using NetworkX where each pair is a node and two nodes are connected with the same rule reported above, still with d=0.08. If the algorithm at step 1 takes too long, repeat step 1. Note that here d=0.08 (and not 0.8 as in the previous item), as in this way the resulting graph is sparser.

## Set up variables

In [12]:
radius = 0.08
R = nx.Graph()
xMin = 30
xMax = 50
yMin = 10
yMax = 20
couples_count = 2000
points_dataframe = pd.DataFrame()

# Generate column x with couples_count rows of elements in [xMin, xMax)
points_dataframe['x'] = np.random.random_sample(couples_count) * (xMax - xMin) + xMin

# Generate column y with couples_count rows of elements in [yMin, yMax)
points_dataframe['y'] = np.random.random_sample(couples_count) * (yMax - yMin) + yMin

# Generate column label with couples_count rows of (x value, y value)
points_dataframe['label'] = "(" + points_dataframe['x'].astype(str) + ", " + points_dataframe['x'].astype(str) + ")"
points_dataframe = points_dataframe[['label', 'x', 'y']]

# Replace duplicates to have clean data
# We just change y value and check that new couple is unique
for dup_ind in points_dataframe[points_dataframe.duplicated()].index:
    while (points_dataframe['label'].value_counts()[points_dataframe.loc[dup_ind, 'label']]) > 1:
        points_dataframe.loc[dup_ind, 'y'] = np.random.random_sample() * (yMax - yMin) + yMin
        points_dataframe.loc[dup_ind, 'label'] = "(" + str(points_dataframe.loc[dup_ind, 'x']) + \
          ", " + str(points_dataframe.loc[dup_ind, 'y']) + ")"

In [13]:
points_dataframe

,label,x,y
0,"(46.548328809759255, 46.548328809759255)",46.548329,19.824551
1,"(32.56775347721311, 32.56775347721311)",32.567753,17.371598
2,"(33.88702765255466, 33.88702765255466)",33.887028,10.063531
3,"(46.724556880978554, 46.724556880978554)",46.724557,10.128249
4,"(46.405501607583304, 46.405501607583304)",46.405502,13.252639
...,...,...,...
1995,"(46.39532706755283, 46.39532706755283)",46.395327,13.908603
1996,"(47.09793985494326, 47.09793985494326)",47.097940,10.089909
1997,"(36.12357947541021, 36.12357947541021)",36.123579,13.082763
1998,"(33.354969381879656, 33.354969381879656)",33.354969,15.756303


## Create Graph using algorithm # 1

In [16]:
#%%timeit
#P_all_couples.clear()
#allCoupleEdges(P_all_couples, points_dataframe, radius)
print('Cell commented because of high time consuming, executed just once with result:')
print('1min 25s ± 234 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)')

Cell commented because of high time consuming, executed just once with result:
1min 25s ± 234 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Create Graph using algorithm # 2

In [15]:
%%timeit
R.clear()
binarySearchEdges(R, points_dataframe, radius)

1.25 s ± 6.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# 3. Weight graphs
Both P and R can be seen as weighted graphs putting on each edge the distance between the two cities. Modify P and R to weight their edges.

### Utility function
Given a graph and a dataframe containing all nodes of the graph with:
 - Column 0 $\rightarrow$ ID
 - Column 1 $\rightarrow$ X Position  
 - Column 2 $\rightarrow$ Y Position  
 
adds weight (distance) to each edge of the graph 

In [97]:
def weightGraph(graph, dataframe):
    for edge in graph.edges:
        graph.edges[edge]['weight'] = mt.sqrt(((dataframe.loc[dataframe.iloc[:, 0] == edge[0]].iloc[0, 1]) - \
                                         (dataframe.loc[dataframe.iloc[:, 0] == edge[1]].iloc[0, 1])) ** 2 +\
                                        ((dataframe.loc[dataframe.iloc[:, 0] == edge[0]].iloc[0, 2]) - \
                                         (dataframe.loc[dataframe.iloc[:, 0] == edge[1]].iloc[0, 2])) ** 2)

In [98]:
weightGraph(P, city_dataframe)
weightGraph(R, points_dataframe)

# 4. Eulerian Path

From Wikipedia (https://en.wikipedia.org/wiki/Eulerian_path):
In graph theory, an Eulerian trail (or Eulerian path) is a trail in a finite graph that visits every edge exactly once (allowing for revisiting vertices). Similarly, an Eulerian circuit or Eulerian cycle is an Eulerian trail that starts and ends on the same vertex.

## Fleury's algorithm
Fleury's algorithm is an elegant but inefficient algorithm.
 1. Check that graph has all edges in the same component
 2. Check that graph has at most 2 vertices of odd degree
 3. Choose a vertex of odd degree, if the graph has none choose an arbitrary vertex.  
    3.1 Choose next edge in the path to be one whose deletion would not disconnect the graph. If there is no such edge pick the remaining edge left at the current vertex.  
    3.2 Use this edge to reach the other node and delete the edge.  
    3.3 If current vertex has no more edges it means that the Graph has no more edge. Otherwise return to 3.1
    
While the graph traversal in Fleury's algorithm is linear in the number of edges, i.e. $\mathcal{O}\ (\ |E|\ )$, we also need to factor in the complexity of detecting bridges.  
If we are to re-run Tarjan's linear time bridge-finding algorithm after the removal of every edge, Fleury's algorithm will have a time complexity of $\mathcal{O}\ (\ |E|^2\ )$.  
A dynamic bridge-finding algorithm of Thorup allows this to be improved to $\mathcal{O}\ (\ |E|\ \cdot\ (\ log\ ⁡|E|\ )^3\ log\ log\ |E|\ )$, but this is still significantly slower than alternative algorithms. 

In [ ]:
def fleury(graph):
    # Check if graph is connected
    if ! nx.is_connected(graph):
        raise Exception("Graph is not connected")
        
    # Check which nodes have odd degree and raise exception if more than 2 have been found
    odd_degree_nodes = []
    for node in graph.nodes:
        if graph.degree([node]) % 2 != 0:
            if len(odd_degree_nodes) == 2:
                raise Exception("Graph has more than 2 nodes with odd degree")
            
            odd_degree_nodes.append(node)
    
    # Start with a node with odd degree (if any)
    if len(odd_degree_nodes) > 0:
        node = odd_degree_nodes[0]
    else:
        node = [*graph.nodes][0]
    
    end = False
    trail = []
    while !end:
        
        # Search edge to node that belong to the same connected component. 
        # I.e. edge between those 2 nodes is not a bridge 
        # --> its deletion will not make graph disconnected
        not_bridge_neigh = neigh in nx.node_connected_component(P, node) & {*P[node]}
        if len(not_bridge_neigh) > 0:
            next_node = not_bridge_neigh.pop()
                
        # If node has at least 1 neighbour
        elif len(P[node]) > 0:
            next_node = [*P[node]][0]
            
        else:
            end = True
            
        trail.append(node)
        graph.remove_node(node)
        node = next_node

In [185]:
P.get_edge_data('Siracusa', 'Ragusa')['weight']
[*P['Siracusa']]
P.*?

## Hierholzer's algorithm
Hierholzer's is more efficient than Fleury's algorithm:
  1. Choose any starting vertex v, and follow a trail of edges from that vertex until returning to v. It is not possible to get stuck at any vertex other than v, because the even degree of all vertices ensures that, when the trail enters another vertex w there must be an unused edge leaving w. The tour formed in this way is a closed tour, but may not cover all the vertices and edges of the initial graph.
    As long as there exists a vertex u that belongs to the current tour but that has adjacent edges not part of the tour, start another trail from u, following unused edges until returning to u, and join the tour formed in this way to the previous tour.
    Since we assume the original graph is connected, repeating the previous step will exhaust all edges of the graph.

By using a data structure such as a doubly linked list to maintain the set of unused edges incident to each vertex, to maintain the list of vertices on the current tour that have unused edges, and to maintain the tour itself, the individual operations of the algorithm (finding unused edges exiting each vertex, finding a new starting vertex for a tour, and connecting two tours that share a vertex) may be performed in constant time each, so the overall algorithm takes linear time, O ( | E | ) {\displaystyle O(|E|)} O(|E|).[8]

This algorithm may also be implemented with a queue. Because it is only possible to get stuck when the queue represents a closed tour, one should rotate the queue (remove an element from the head and add it to the tail) until unstuck, and continue until all edges are accounted for. This also takes linear time, as the number of rotations performed is never larger than | E | {\displaystyle |E|} |E|. 

# Todo
 - connected manual check
 - odd degree check

#dLong = dataframe.sort_values(by='long')

#datas = pd.DataFrame(d)

# O(n) Cleaning data
# Removing unusable latitude / longitude
#for i in range(len(d)):
#    if d[i]['lat'] != 0 and d[i]['long'] != 0:
#        dLong.append(d[i])

# O(n log n)
#dLong.sort(key=sortLongFun)

#dLat = dLong.copy()

# O(n log n)
#dLat.sort(key=sortLatFun)  



import pixiedust
import pdb

def sortLatFun(elem):
    return elem['lat']

def sortLongFun(elem):
    return elem['long']

def my_print(d):
    for i in range(len(d)):
        display(d[i]['lat'])
        
'''
def binarySearch(list, item_index):
    first = 0
    last = len(list)-1

    while first <= last:
        midpoint = (first + last)//2
        if item_index
           list[item_index]['lat'] - radius <= list[midpoint]['lat'] and \
           list[item_index]['lat'] + radius >= list[midpoint]['lat'] and \
           list[item_index]['long'] - radius <= list[midpoint]['long'] and \
           list[item_index]['long'] + radius >= list[midpoint]['long']:
            
            
        else:
            if item < list[midpoint]:
                last = midpoint-1
            else:
                first = midpoint+1

    return found
'''
'''
def binarySearch(d, item, item_index):
    if len(d) == 0:
        return
    
    midpoint = len(d) // 2
    HEAD = "len:" + str(len(d)) + ", midpoint: " + str(midpoint) + ", item_lat: " + str(item['lat']) + ", midpoint_lat: " + str(d[midpoint]['lat'])
    my_print(d)

    if item['lat'] - radius <= d[midpoint]['lat'] and \
       item['lat'] + radius >= d[midpoint]['lat']:
        if len(d) >= midpoint and midpoint != 0:
            display(HEAD + " - Calling: d[:" + str(midpoint) + "] ")
            binarySearch(d[:midpoint], item, item_index)
        
        if len(d) > midpoint+1:
            display(HEAD + " - Calling: d[" + str(midpoint+1) + ":] ")
            binarySearch(d[midpoint+1:], item, item_index)
            
        if item['long'] - radius <= d[midpoint]['long'] and \
           item['long'] + radius >= d[midpoint]['long'] and \
           item != d[midpoint]:
            display(HEAD + " - Adding edge")
            P.add_edge(midpoint, item_index)
    else:
        if item['lat'] < d[midpoint]['lat'] and \
           len(d) >= midpoint and \
           midpoint != 0:
            display(HEAD + " - Else Calling: d[:" + str(midpoint) + "] ")
            binarySearch(d[:midpoint], item, item_index)
        elif len(d) > midpoint + 1:
                display(HEAD + " - Else Calling: d[" + str(midpoint+1) + ":] ")
                binarySearch(d[midpoint+1:], item, item_index)
'''
'''
def binarySearch(d, item):
    midpoint = len(d)//2
    HEAD = "len:" + str(len(d)) + ", midpoint " + str(midpoint) + ", " + str(item['lat']) + ", " + str(d[midpoint]['lat'])
    display(HEAD)
    if item['lat'] - radius <= d[midpoint]['lat'] and \
       item['lat'] + radius >= d[midpoint]['lat']:
        if len(d) >= midpoint:
            display(HEAD + " - Calling: d[:" + str(midpoint) + "] ")
            binarySearch(d[:midpoint], item)
        
        if len(d) > midpoint+1:
            display(HEAD + " - Calling: d[" + str(midpoint+1) + ":] ")
            binarySearch(d[midpoint+1:], item)
            
        if item['long'] - radius <= d[midpoint]['long'] and \
           item['long'] + radius >= d[midpoint]['long'] and \
           item != midpoint:
            display(HEAD + " - Adding edge")
            P.add_edge(midpoint, item)
    else:
        if item['lat'] < d[midpoint]['lat'] and len(d) >= midpoint:
            display(HEAD + " - Else Calling: d[:" + str(midpoint) + "] ")
            binarySearch(d[:midpoint], item)
        elif len(d) > midpoint + 1:
                display(HEAD + " - Else Calling: d[" + str(midpoint+1) + ":] ")
                binarySearch(d[midpoint+1:], item)
'''                 

import random
display(len(d))
d = random.sample(d, 1500)